<a href="https://colab.research.google.com/github/kanovotn/sentiment-analysis-model-trainer/blob/master/notebooks/inference_and_evaluation_sentiment_analysis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference and evaluation of sentiment analysis model

Find the full decription of this project in my blog: https://kanovotn.github.io/2024-05-01-sentiment-analysis-with-hugging-face/

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# Inference on test "imdb" dataset

In [7]:
from datasets import load_dataset, concatenate_datasets


def load_and_split_dataset(dataset_name):
        # Split ratios
        train_split = 0.6
        test_split = 0.2

        # Load the dataset
        dataset_train = load_dataset(dataset_name, split="train")
        dataset_test = load_dataset(dataset_name, split="test")

        # Merge them and shuffle
        dataset_full = concatenate_datasets([dataset_train, dataset_test])
        # Shuffle the data with fixed seed to ensure the reproducibility of the dataset
        dataset_full = dataset_full.shuffle(seed=42).flatten_indices()

        # Calculate the number of samples for train, validate, and test
        total_samples = len(dataset_full)
        train_size = int(total_samples * train_split)
        test_size = int(total_samples * test_split)

        # Split the dataset
        dataset_train = dataset_full.select(range(train_size))
        dataset_test = dataset_full.select(range(train_size, train_size + test_size))
        dataset_validation = dataset_full.select(range(train_size + test_size, total_samples))

        return dataset_train, dataset_validation, dataset_test

In [8]:
import pandas as pd
from datasets import load_dataset
from evaluate import evaluator
from transformers import AutoTokenizer

models = [
    "lyrisha/distilbert-base-finetuned-imdb-sentiment",
    "lyrisha/distilbert-base-finetuned-sentiment",
    "distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    "siebert/sentiment-roberta-large-english"
]

#data = load_dataset("imdb", split="test[80%:]")

dataset_train, dataset_validation, dataset_test = load_and_split_dataset("imdb")
#tokenizer = AutoTokenizer.from_pretrained("lyrisha/distilbert-base-finetuned-imdb-sentiment")
#def tokenize_function(example):
#  return tokenizer(example['text'], max_length=512, truncation=True, padding="max_length")

#tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)


task_evaluator = evaluator("sentiment-analysis")

results = []
for model in models:
    results.append(
        task_evaluator.compute(
            model_or_pipeline=model, data=dataset_test, label_mapping={"NEGATIVE": 0, "POSITIVE": 1}
            )
        )

df = pd.DataFrame(results, index=models)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 50000
})


Flattening the indices:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

# Results on IMDB test set

In [9]:
df

,accuracy,total_time_in_seconds,samples_per_second,latency_in_seconds
lyrisha/distilbert-base-finetuned-imdb-sentiment,0.9257,72.652311,137.641872,0.007265
lyrisha/distilbert-base-finetuned-sentiment,0.9092,72.201125,138.501997,0.007220
distilbert/distilbert-base-uncased-finetuned-sst-2-english,0.8868,72.295551,138.321098,0.007230
siebert/sentiment-roberta-large-english,0.9497,309.411738,32.319394,0.030941


# Inference on test "rotten_tomatoes" dataset

In [3]:
import pandas as pd
from datasets import load_dataset
from evaluate import evaluator
from transformers import pipeline

# List of model IDs
models = [
    "lyrisha/distilbert-base-finetuned-imdb-sentiment",
    "lyrisha/distilbert-base-finetuned-sentiment",
    "distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    "siebert/sentiment-roberta-large-english"
]

# Load the test data from the Rotten Tomatoes dataset
data = load_dataset("rotten_tomatoes", split="test")

# Initialize the evaluator for sentiment analysis
task_evaluator = evaluator("sentiment-analysis")

results = []
for model in models:
    # Use the model to compute results on the dataset
    model_results = task_evaluator.compute(
        model_or_pipeline=model, data=data, label_mapping={"NEGATIVE": 0, "POSITIVE": 1}
    )
    results.append(model_results)

df3 = pd.DataFrame(results, index=models)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

# Results on Rotten tomatoes test set

In [4]:
df3

,accuracy,total_time_in_seconds,samples_per_second,latency_in_seconds
lyrisha/distilbert-base-finetuned-imdb-sentiment,0.822702,7.125400,149.605642,0.006684
lyrisha/distilbert-base-finetuned-sentiment,0.880863,107.658127,9.901714,0.100993
distilbert/distilbert-base-uncased-finetuned-sst-2-english,0.896811,6.529148,163.267869,0.006125
siebert/sentiment-roberta-large-english,0.920263,21.053129,50.633803,0.019750
